<a href="https://colab.research.google.com/github/Dije-7/NLP/blob/main/TransformerNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('cfilt/iitb-english-hindi')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [4]:
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']


In [5]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00


In [6]:
from transformers import MarianMTModel, MarianTokenizer

NLP_Eng_to_hin = "Helsinki-NLP/opus-mt-en-hi"

model = MarianMTModel.from_pretrained(NLP_Eng_to_hin)
tokenizer = MarianTokenizer.from_pretrained(NLP_Eng_to_hin)

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
pip install sacremoses


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.5 MB/s eta 0:00:00


In [8]:
train_data = dataset['train'].train_test_split(test_size=0.1)['train']
validation_data = dataset['train'].train_test_split(test_size=0.1)['test']

In [ ]:
import torch

def process_data(data):
    source_sentences = []
    target_sentences = []
    for sample in data:
        if isinstance(sample, dict):  # Check if the chunk is a dictionary
            source_sentences.append(sample['translation']['en'])
            target_sentences.append(sample['translation']['hi'])
    return source_sentences, target_sentences

# Split data into smaller chunks and process
batch_size = 1000  # Define the batch size

# Process training data
train_source_chunks = [train_data[i:i + batch_size] for i in range(0, len(train_data), batch_size)]
train_target_chunks = [validation_data[i:i + batch_size] for i in range(0, len(validation_data), batch_size)]

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 5  # Define the number of epochs

for epoch in range(epochs):
    model.train()
    for src_chunk, tgt_chunk in zip(train_source_chunks, train_target_chunks):
        train_source_sentences, train_target_sentences = process_data(src_chunk), process_data(tgt_chunk)

        # Check if lists are empty
        if not train_source_sentences[0] or not train_target_sentences[0]:
            continue

        train_input_ids = tokenizer(train_source_sentences[0], return_tensors="pt", padding=True, truncation=True)
        train_labels = tokenizer(train_target_sentences[0], return_tensors="pt", padding=True, truncation=True)

        for i in range(1, len(train_source_sentences)):
            if not train_source_sentences[i] or not train_target_sentences[i]:
                continue

            temp_input = tokenizer(train_source_sentences[i], return_tensors="pt", padding=True, truncation=True)
            temp_label = tokenizer(train_target_sentences[i], return_tensors="pt", padding=True, truncation=True)
            train_input_ids['input_ids'] = torch.cat((train_input_ids['input_ids'], temp_input['input_ids']))
            train_labels['input_ids'] = torch.cat((train_labels['input_ids'], temp_label['input_ids']))

        optimizer.zero_grad()
        outputs = model(**train_input_ids, labels=train_labels.input_ids)
        loss = loss_fn(outputs.logits.view(-1, outputs.logits.shape[-1]), train_labels.input_ids.view(-1))
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}: Loss - {loss.item()}")

    # Validation steps
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_sample in validation_data:
            val_source = val_sample['translation']['en']
            val_target = val_sample['translation']['hi']
            val_input = tokenizer(val_source, return_tensors="pt", padding=True, truncation=True)
            val_label = tokenizer(val_target, return_tensors="pt", padding=True, truncation=True)

            outputs = model(**val_input, labels=val_label.input_ids)
            val_loss += loss_fn(outputs.logits.view(-1, outputs.logits.shape[-1]), val_label.input_ids.view(-1)).item()

    average_val_loss = val_loss / len(validation_data)
    print(f"Validation Loss: {average_val_loss}")

In [ ]:
# import torch

# train_source_sentences = [sample['translation']['en'] for sample in train_data]
# train_target_sentences = [sample['translation']['hi'] for sample in train_data]

# val_source_sentences = [sample['translation']['en'] for sample in validation_data]
# val_target_sentences = [sample['translation']['hi'] for sample in validation_data]

# train_input_ids = tokenizer(train_source_sentences, return_tensors="pt", padding=True, truncation=True)
# train_labels = tokenizer(train_target_sentences, return_tensors="pt", padding=True, truncation=True)

# val_input_ids = tokenizer(val_source_sentences, return_tensors="pt", padding=True, truncation=True)
# val_labels = tokenizer(val_target_sentences, return_tensors="pt", padding=True, truncation=True)

# # Define optimizer and loss function
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
# loss_fn = torch.nn.CrossEntropyLoss()

# # Training loop (example)
# epochs = 5
# for epoch in range(epochs):
#     optimizer.zero_grad()
#     outputs = model(**train_input_ids, labels=train_labels.input_ids)
#     loss = loss_fn(outputs.logits.view(-1, outputs.logits.shape[-1]), train_labels.input_ids.view(-1))
#     loss.backward()
#     optimizer.step()
#     print(f"Epoch {epoch + 1}: Loss - {loss.item()}")

# # Validation (example)
# with torch.no_grad():
#     val_outputs = model(**val_input_ids, labels=val_labels.input_ids)
#     val_loss = loss_fn(val_outputs.logits.view(-1, val_outputs.logits.shape[-1]), val_labels.input_ids.view(-1))
#     print(f"Validation Loss: {val_loss.item()}")

In [ ]:
metric = load_metric("sacrebleu")
references = test_target_sentences
predictions = translated_sentences
bleu_score = metric.compute(predictions=predictions, references=references)

print(f"BLEU Score: {bleu_score['score']}")

In [ ]:
input_text = "Hello! How are you?"
translated_text = model.generate(**tokenizer(input_text, return_tensors="pt"))
print(tokenizer.decode(translated_text[0], skip_special_tokens=True))

In [ ]:
# input sentences
input_text = ["Elon Musk sells $8.5 billion in Tesla stock",
              "I'm a professional academic and research writer.",
              "Get a job in US and work in Germany"]

# convert sentences to tensors
model_inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)